<a href="https://colab.research.google.com/github/nattaran/health-tequity-case-nasrin/blob/main/VoicePipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Mount Google Drive**

In [6]:
# from google.colab import drive
# drive.mount('/content/drive')


Mounted at /content/drive


**Set Working Directory**

In [8]:
# import os

# # Update this path to match your Google Drive folder
# BASE_DIR = '/content/drive/MyDrive/health-tequity-case'
# os.chdir(BASE_DIR)

# # Verify
# print(f"✅ Current directory: {os.getcwd()}")
# print(f"📁 Files in directory: {os.listdir('.')}")

**install Required Packages**














In [ ]:
# Install packages
# !pip install -q openai-whisper
# !pip install -q git+https://github.com/openai/whisper.git
# !pip install -q librosa soundfile
# !pip install -q deep-translator
# !pip install -q gtts
# !pip install -q jiwer

# print("✅ All packages installed successfully!")

**Import Libraries **

In [2]:
# import whisper
# import os
# import pandas as pd
# import numpy as np
# import json
# from datetime import datetime, timedelta
# from deep_translator import GoogleTranslator
# from gtts import gTTS
# import jiwer
# from jiwer import wer, cer
# from jiwer import process_words
# import warnings
# import torch
# warnings.filterwarnings('ignore')


# print("✅ All libraries imported successfully!")

# # Update this path to match your Google Drive folder


**Accessing to the Spanish Audio Files **

LLM **part**

In [3]:
# !pip -q install transformers accelerate bitsandbytes sentencepiece einops safetensors



In [4]:
# Optional: see what Colab gave you
# !nvidia-smi || true

# import sys, platform
# print("Python:", sys.version)
# print("Platform:", platform.platform())

In [7]:
# !pip -q install "openai==1.51.2" "httpx==0.27.2" "httpcore==1.0.5" pandas


In [9]:
# from google.colab import userdata
# api_key = userdata.get("OPENAI_API_KEY")
# if not api_key:
#     raise RuntimeError("Add OPENAI_API_KEY in the Secrets panel (left sidebar, key icon).")

In [7]:
# Upload the CSV from google drive

# from google.colab import drive
# drive.mount('/content/drive')
csv_path = "/content/drive/MyDrive/health-tequity-case/Data/synthetic_bp_one_person.csv"

# Mount Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install Required Packages

In [8]:
!pip install -r "/content/drive/MyDrive/health-tequity-case/requirements.txt"



  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-sjy7cusj
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-sjy7cusj
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.7 MB/s eta 0:00:00
   ━━━━

# Load OpenAI API Key (From secretes)

In [44]:
from google.colab import userdata
api_key = userdata.get("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("Add OPENAI_API_KEY in the Secrets panel (left sidebar, key icon).")

In [48]:
!find /content/drive/MyDrive/health-tequity-case -name "synthetic_bp_one_person.csv"


/content/drive/MyDrive/health-tequity-case/Data/BloodPressure/synthetic_bp_one_person.csv


# Define Paths and Create Folders

In [49]:
import json, re, pandas as pd, whisper, Levenshtein
from openai import OpenAI
from jiwer import wer, mer, wil, process_words
import warnings
warnings.filterwarnings("ignore")
import os

BASE_PATH = "/content/drive/MyDrive/health-tequity-case"

# --- Define key folders ---
AUDIO_INPUT_FOLDER = os.path.join(BASE_PATH, "Input_Audio_Files")        # Spanish question audio files
AUDIO_OUTPUT_FOLDER = os.path.join(BASE_PATH, "Data", "audio_out")       # Spanish TTS answers
CSV_OUTPUT_FOLDER = os.path.join(BASE_PATH, "Data", "csv_results")       # WER, CER, SER + pipeline outputs
BP_DATA_FOLDER = os.path.join(BASE_PATH, "Data", "BloodPressure")        # Blood pressure dataset

# --- Create required folders if they don’t exist ---
for folder in [AUDIO_OUTPUT_FOLDER, CSV_OUTPUT_FOLDER, BP_DATA_FOLDER]:
    os.makedirs(folder, exist_ok=True)

# --- Validate Input Audio Folder ---
if not os.path.exists(AUDIO_INPUT_FOLDER):
    raise FileNotFoundError(f"❌ Input folder not found: {AUDIO_INPUT_FOLDER}")

# --- Collect available audio files ---
audio_files = [f for f in os.listdir(AUDIO_INPUT_FOLDER) if f.lower().endswith(('.wav', '.mp3', '.m4a'))]
if not audio_files:
    raise ValueError(f"❌ No audio files found in {AUDIO_INPUT_FOLDER}")

print(f"✅ Found {len(audio_files)} Spanish audio file(s): {audio_files}")

# --- Blood Pressure dataset check ---
csv_path = os.path.join(BP_DATA_FOLDER, "synthetic_bp_one_person.csv")

if not os.path.exists(csv_path):
    print(f"⚠️ Blood pressure dataset not found at:\n   {csv_path}")
    print("👉 Please upload your synthetic_bp_one_person.csv to this folder before running the pipeline.")
else:
    print(f"✅ Found blood pressure dataset: {csv_path}")

# --- Initialize OpenAI client ---
client = OpenAI(api_key=api_key)
print("✅ OpenAI client initialized successfully.")


✅ Found 6 Spanish audio file(s): ['q2_es.wav', 'q1_es.wav', 'q4_es.wav', 'q3_es.wav', 'q6_es.wav', 'q5_es.wav']
✅ Found blood pressure dataset: /content/drive/MyDrive/health-tequity-case/Data/BloodPressure/synthetic_bp_one_person.csv
✅ OpenAI client initialized successfully.


# ASR Transcrioption Generation Using openAI Whisper Model
**Audio -> Transcription -> English Transcription**

In [50]:
def transcribe_spanish_audio(model, audio_path):
    print(f"🎧 Transcribing: {audio_path}")
    result = model.transcribe(audio_path, language="spanish", task="transcribe", verbose=False)
    return result["text"].strip(), result["language"]

def translate_spanish_to_english(spanish_text: str) -> str:
    """Translate Spanish transcription to English."""
    prompt = f"Translate the following Spanish medical question into clear English:\n\n{spanish_text}"
    result = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    return result.choices[0].message.content.strip()

def process_and_translate_audio(audio_folder, audio_files, output_csv):
    model = whisper.load_model("base")
    all_results = []

    print("\n🎯 STARTING SPANISH TRANSCRIPTION + TRANSLATION\n" + "="*60)
    for i, audio_file in enumerate(audio_files, 1):
        audio_path = os.path.join(audio_folder, audio_file)
        if not os.path.exists(audio_path):
            print(f"⚠️ {audio_file} not found, skipping...")
            continue

        spanish_text, detected_lang = transcribe_spanish_audio(model, audio_path)
        english_text = translate_spanish_to_english(spanish_text)

        all_results.append({
            "audio_file": audio_file,
            "spanish_transcription": spanish_text,
            "english_translation": english_text,
            "language_detected": detected_lang
        })

        print(f"\n[{i}] {audio_file}")
        print(f"🇪🇸 {spanish_text}")
        print(f"🇬🇧 {english_text}")

    df = pd.DataFrame(all_results)
    df.to_csv(output_csv, index=False)
    print(f"\n✅ Transcriptions + translations saved to {output_csv}")
    return df


# ASR Evaluation (WER, CER, SER)


In [51]:
def compute_cer(reference: str, hypothesis: str) -> float:
    reference, hypothesis = reference.strip(), hypothesis.strip()
    if not reference:
        return 1.0 if hypothesis else 0.0
    return Levenshtein.distance(reference, hypothesis) / len(reference)

def compute_sentence_error(reference: str, hypothesis: str) -> int:
    return 0 if reference.strip() == hypothesis.strip() else 1

def evaluate_asr_performance(ground_truth_csv, transcribed_csv, output_csv):
    gt_df = pd.read_csv(ground_truth_csv)
    tr_df = pd.read_csv(transcribed_csv)
    gt_df.columns = [c.lower().strip() for c in gt_df.columns]
    tr_df.columns = [c.lower().strip() for c in tr_df.columns]
    df = pd.merge(gt_df, tr_df, on="audio_file", how="inner")

    results = []
    print(f"\n🎯 Evaluating {len(df)} files for ASR performance...\n")
    for _, row in df.iterrows():
        ref, hyp = str(row["ground_truth"]), str(row["spanish_transcription"])
        m = process_words(ref, hyp)
        wer_score = round(m.wer, 4)
        subs, dels, ins = m.substitutions, m.deletions, m.insertions
        cer = round(compute_cer(ref, hyp), 4)
        ser = compute_sentence_error(ref, hyp)
        results.append({
            "audio_file": row["audio_file"],
            "WER": wer_score, "Substitutions": subs,
            "Deletions": dels, "Insertions": ins,
            "CER": cer, "SER": ser
        })
        print(f"🎧 {row['audio_file']} → WER: {wer_score}, CER: {cer}, SER: {ser}")

    out_df = pd.DataFrame(results)
    out_df.to_csv(output_csv, index=False)
    print(f"\n✅ ASR metrics saved to: {output_csv}")
    return out_df

# *GPT Data Analysis + Translation + TTS*

In [52]:
# ================================================================
# 3️⃣ GPT DATA ANALYSIS + TRANSLATION + TTS
# ================================================================
SYSTEM = """
You are a careful data analyst.
You receive a synthetic blood pressure dataset with columns: date, age, sex, systolic, diastolic.
Do ALL analysis yourself using ONLY the CSV provided.
Answer questions like: daily readings, averages, trends, comparisons, etc.
Return JSON:
{ "answer": "<English answer>", "computed_fields": { "numeric values used" } }
"""

def ask_gpt(question_en, csv_block):
    user = f"CSV data:\n{csv_block}\n\nQUESTION:\n{question_en}"
    resp = client.chat.completions.create(
        model="gpt-4o",
        temperature=0,
        messages=[{"role": "system", "content": SYSTEM}, {"role": "user", "content": user}]
    ).choices[0].message.content
    clean = re.sub(r"^```json|```$", "", resp.strip(), flags=re.M | re.I)
    start, end = clean.find("{"), clean.rfind("}")
    return json.loads(clean[start:end+1])

def translate_to_spanish(english_text):
    prompt = f"Translate this English medical answer into clear, neutral Spanish:\n{english_text}"
    return client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    ).choices[0].message.content.strip()

def text_to_speech_spanish(text, filename, voice="alloy"):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with client.audio.speech.with_streaming_response.create(
        model="gpt-4o-mini-tts", voice=voice, input=text
    ) as response:
        response.stream_to_file(filename)
    print(f"🔊 Saved Spanish audio: {filename}")
    return filename


# Main PIPELINE

In [55]:
# ================================================================
# 4️⃣ MAIN PIPELINE
# ================================================================
def run_full_pipeline(csv_path, audio_folder, audio_files):
    # Step 1 — Transcribe and Translate Spanish Audio
    trans_csv = os.path.join(CSV_OUTPUT_FOLDER, "audio_translations.csv")
    trans_df = process_and_translate_audio(audio_folder, audio_files, trans_csv)

    # Step 2 — Evaluate ASR (WER, CER, SER)
    gt_csv = os.path.join(audio_folder, "ground_truth.csv")
    asr_csv = os.path.join(CSV_OUTPUT_FOLDER, "asr_metrics.csv")
    asr_df = evaluate_asr_performance(gt_csv, trans_csv, asr_csv)

    # Step 3 — Load Blood Pressure Data
    df_bp = pd.read_csv(csv_path)
    csv_block = df_bp.to_csv(index=False)

    results = []
    for i, row in trans_df.iterrows():
        q_num = i + 1
        q_en = row["english_translation"]
        print(f"\n🔹 Q{q_num}: {q_en}")

        try:
            ans = ask_gpt(q_en, csv_block)
            ans_en = ans.get("answer", "").strip()
            ans_es = translate_to_spanish(ans_en)

            audio_file = os.path.join(AUDIO_OUTPUT_FOLDER, f"answer_{q_num}_es.wav")
            text_to_speech_spanish(ans_es, audio_file)

            results.append({
                "question_number": q_num,
                "audio_file_in": row["audio_file"],
                "spanish_question": row["spanish_transcription"],
                "english_question": q_en,
                "english_answer": ans_en,
                "spanish_answer": ans_es,
                "audio_answer_file": audio_file,
                "computed_fields": json.dumps(ans.get("computed_fields", {}))
            })
            print(f"✅ Completed Q{q_num}")

        except Exception as e:
            print(f"❌ Error Q{q_num}: {e}")

    # Step 4 — Save Final Results
    final_csv = os.path.join(CSV_OUTPUT_FOLDER, "final_pipeline_results.csv")
    pd.DataFrame(results).to_csv(final_csv, index=False)
    print(f"\n✅ All results saved to {final_csv}")
    return results

In [58]:
# ================================================================
# 5️⃣ RUN
# ================================================================
csv_path = os.path.join(BASE_PATH, BP_DATA_FOLDER, "synthetic_bp_one_person.csv")
run_full_pipeline(csv_path, AUDIO_INPUT_FOLDER, audio_files)



🎯 STARTING SPANISH TRANSCRIPTION + TRANSLATION
🎧 Transcribing: /content/drive/MyDrive/health-tequity-case/Input_Audio_Files/q2_es.wav


100%|██████████| 470/470 [00:01<00:00, 239.29frames/s]



[1] q2_es.wav
🇪🇸 ¿Cuáles fueron mis valores de presión arterial durante la última semana?
🇬🇧 What were my blood pressure readings over the last week?
🎧 Transcribing: /content/drive/MyDrive/health-tequity-case/Input_Audio_Files/q1_es.wav


100%|██████████| 458/458 [00:02<00:00, 224.61frames/s]



[2] q1_es.wav
🇪🇸 ¿Cuáles son mis presiones arteriales histólica y diastólica hoy?
🇬🇧 What are my systolic and diastolic blood pressures today?
🎧 Transcribing: /content/drive/MyDrive/health-tequity-case/Input_Audio_Files/q4_es.wav


...

100%|██████████| 1574/1574 [00:03<00:00, 461.74frames/s]



[5] q6_es.wav
🇪🇸 ¿En qué día mi presión arterial excedió los niveles normales? Compare mi presión arterial promedio en la primera semana y la última semana de este mes. ¿Cuál fue mi presión arterial diastólica más baja este mes?
🇬🇧 What day did my blood pressure exceed normal levels? Compare my average blood pressure in the first week and the last week of this month. What was my lowest diastolic blood pressure this month?
🎧 Transcribing: /content/drive/MyDrive/health-tequity-case/Input_Audio_Files/q5_es.wav


100%|██████████| 328/328 [00:01<00:00, 174.11frames/s]



[6] q5_es.wav
🇪🇸 ¿Cuál era mi presión arterial el 10 de octubre?
🇬🇧 What was my blood pressure on October 10th?

✅ Transcriptions + translations saved to /content/drive/MyDrive/health-tequity-case/Data/csv_results/audio_translations.csv

🎯 Evaluating 6 files for ASR performance...

🎧 q1_es.wav → WER: 0.1111, CER: 0.0156, SER: 1
🎧 q2_es.wav → WER: 0.0, CER: 0.0, SER: 0
🎧 q3_es.wav → WER: 0.0, CER: 0.0, SER: 0
🎧 q4_es.wav → WER: 0.1, CER: 0.0175, SER: 1
🎧 q5_es.wav → WER: 0.0, CER: 0.0, SER: 0
🎧 q6_es.wav → WER: 0.0, CER: 0.0, SER: 0

✅ ASR metrics saved to: /content/drive/MyDrive/health-tequity-case/Data/csv_results/asr_metrics.csv

🔹 Q1: What were my blood pressure readings over the last week?
🔊 Saved Spanish audio: /content/drive/MyDrive/health-tequity-case/Data/audio_out/answer_1_es.wav
✅ Completed Q1

🔹 Q2: What are my systolic and diastolic blood pressures today?
🔊 Saved Spanish audio: /content/drive/MyDrive/health-tequity-case/Data/audio_out/answer_2_es.wav
✅ Completed Q2

🔹 Q3: 

[{'question_number': 1,
  'audio_file_in': 'q2_es.wav',
  'spanish_question': '¿Cuáles fueron mis valores de presión arterial durante la última semana?',
  'english_question': 'What were my blood pressure readings over the last week?',
  'english_answer': 'Over the last week, from October 10, 2025, to October 16, 2025, your blood pressure readings were as follows: \n- October 10: 160/101 mmHg (hypertensive)\n- October 11: 152/94 mmHg (hypertensive)\n- October 12: 157/98 mmHg (hypertensive)\n- October 13: 144/100 mmHg (hypertensive)\n- October 14: 145/91 mmHg (hypertensive)\n- October 15: 124/81 mmHg (elevated)\n- October 16: 110/76 mmHg (normal)',
  'spanish_answer': 'Durante la última semana, del 10 de octubre de 2025 al 16 de octubre de 2025, sus lecturas de presión arterial fueron las siguientes:  \n- 10 de octubre: 160/101 mmHg (hipertensiva)  \n- 11 de octubre: 152/94 mmHg (hipertensiva)  \n- 12 de octubre: 157/98 mmHg (hipertensiva)  \n- 13 de octubre: 144/100 mmHg (hipertensiva)

In [ ]:
   ! pip install python-Levenshtein

# **Define Templates + GPT call (JSON output)**